# Tutorial: explain a tabular classifier through LORE

LORE provides interpretable explanations for the decisions made by machine learning models, particularly in situations where it might be challenging to explain the model's behavior using traditional methods. LORE is designed to offer explanations on a per-instance basis, meaning it provides insights into why a specific prediction was made for a particular data point. This is in contrast to global explanations that aim to provide an overview of a model's behavior across the entire dataset.


In this tutorial we will explain the reasoning of a random forest classifier when applied to an instance of the Adult Income dataset ( https://archive.ics.uci.edu/dataset/2/adult). This dataset contains census data from 1994 census database, the classification problem associated to such data involves the prediction of the annual income of a person, given a set of socio-economic caracteristics: 
- working class
- marital status
- occupation
- relationship
- race
- sex
- capital gain/loss
- native country 





### Dataset loading 

Lore libray has a module devoted to dataset handling. Here we use TabularDataset class to load a dataset from a csv file. A TabularDataset has a column that represent the target class (`class_name`) of the classification task  object, and has two main attributes: 
- `df` : a pandas dataframe representing the tabular data
- `descriptor` : a dictionary containing internal data related to the dataset. It is build when a TabularDataset is created, but it could also be edited. It is used by the next steps of LORE methodology to distinguish among numerical, categorical, ordinal and target features of the dataset.

In [1]:
from lore_sa.dataset import TabularDataset
import pandas as pd

dataset = TabularDataset.from_csv('test/resources/adult.csv', class_name = "class")
dataset.df.dropna(inplace = True)
dataset.df


2025-02-10 12:18:03,073 root         INFO     test/resources/adult.csv file imported


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


The `TabularDataset` class does some data inspection and preparation when it is created starting from a text-based file. Each attribute of the data is analysed to derive type and internal statistics. The `descriptor` field is automatically updates with these information. In particular, the `descriptor` is a dictionary that contains the following keys:
- `numerical` : a dictionary containing the numerical features of the dataset and their statistics
- `categorical` : a dictionary containing the categorical features of the dataset and their statistics
- `ordinal` : (this is still not implemented) a dictionary containing the ordinal features of the dataset and their statistics
- `target` : the details of the target feature of the dataset and its statistics  

In [2]:
dataset.descriptor.keys()

dict_keys(['numeric', 'categorical', 'ordinal', 'target'])

Before proceeding with the explanation, let's do some data preprocessing by dropping a few attributes that are not relevant for the learning. 

We operate directly on the `df` attribute of the dataset, then we update the descriptor.


In [3]:
dataset.df.drop(['fnlwgt', 'education-num'], inplace=True, axis=1)
dataset.update_descriptor()

In [17]:
dataset.descriptor

{'numeric': {'age': {'index': 0,
   'min': 17,
   'max': 90,
   'mean': 38.437901995888865,
   'std': 13.134664776855985,
   'median': 37.0,
   'q1': 28.0,
   'q3': 47.0},
  'capital-gain': {'index': 8,
   'min': 0,
   'max': 99999,
   'mean': 1092.0078575691268,
   'std': 7406.346496683503,
   'median': 0.0,
   'q1': 0.0,
   'q3': 0.0},
  'capital-loss': {'index': 9,
   'min': 0,
   'max': 4356,
   'mean': 88.37248856176646,
   'std': 404.29837048637575,
   'median': 0.0,
   'q1': 0.0,
   'q3': 0.0},
  'hours-per-week': {'index': 10,
   'min': 1,
   'max': 99,
   'mean': 40.93123798156621,
   'std': 11.979984229274882,
   'median': 40.0,
   'q1': 40.0,
   'q3': 45.0}},
 'categorical': {'workclass': {'index': 1,
   'distinct_values': ['State-gov',
    'Self-emp-not-inc',
    'Private',
    'Federal-gov',
    'Local-gov',
    'Self-emp-inc',
    'Without-pay'],
   'count': {'State-gov': 1279,
    'Self-emp-not-inc': 2499,
    'Private': 22286,
    'Federal-gov': 943,
    'Local-gov': 20

## Model Learning

We start by training a Random Forest classifier on the Adult dataset. We will use the `df` attribute of the dataset, which is a pandas dataframe containing the tabular data. We will use the `class_name` attribute of the dataset to identify the target feature of the classification task. Since the data contains non-numeric attributes, we proceed with a preprocessing of the data to mange these attributes. We will exploit `Pipeline` class from `sklearn` to create a pipeline that applies one-hot encoding to the categorical features and label encoding to the target feature. 

In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from lore_sa.bbox import sklearn_classifier_bbox


def train_model(dataset: TabularDataset):
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), [0,8,9,10]),
            ('cat', OrdinalEncoder(), [1,2,3,4,5,6,7,11])
        ]
    )
    model = make_pipeline(preprocessor, RandomForestClassifier(n_estimators=100, random_state=42))
    
    X_train, X_test, y_train, y_test = train_test_split(dataset.df.loc[:, 'age':'native-country'].values, dataset.df['class'].values,
                test_size=0.3, random_state=42, stratify=dataset.df['class'].values)
    model.fit(X_train, y_train)
    
    return sklearn_classifier_bbox.sklearnBBox(model)
    

In [5]:
bbox = train_model(dataset)

## Explanation

Given the blackbox model, we can now explain the reasoning behind the classification of a specific instance. We will use the `LORE` class to explain the classification of the first instance of the dataset. The `explain_instance` method of the `LORE` class takes as input the instance to be explained, the blackbox model, and the dataset. It returns an explanation object that contains the explanation of the classification of the instance.

The constructor of the explanator class takes as input the blackbox model and the dataset. The `dataset` instance is used to get the information from the descriptor. In case the trainin set is not available, the descriptor could be built from the test set or by the metadata of the dataset.

In [6]:
from lore_sa.lore import TabularRandomGeneratorLore

tabularLore = TabularRandomGeneratorLore(bbox, dataset)

In [7]:
num_row = 10
x = dataset.df.iloc[num_row][:-1] # we exclude the target feature
# when
explanation = tabularLore.explain(x)
# then
print(explanation)

{'rule': {'premises': [{'attr': 'capital-gain', 'val': 9610.98974609375, 'op': '>'}, {'attr': 'native-country', 'val': 'Iran', 'op': '!='}, {'attr': 'age', 'val': 17.976405143737793, 'op': '>'}], 'consequence': {'attr': 'class', 'val': '>50K', 'op': '='}}, 'counterfactuals': [{'premises': [{'attr': 'capital-gain', 'val': 9610.98974609375, 'op': '>'}, {'attr': 'native-country', 'val': 'Iran', 'op': '='}, {'attr': 'age', 'val': 67.77857208251953, 'op': '>'}, {'attr': 'relationship', 'val': 'Husband', 'op': '='}], 'consequence': {'attr': 'class', 'val': '<=50K', 'op': '='}}]}


### Drill down the explanation process

The example above shows the explanation of the classification of one of the instances of the dataset. The `TabularRandomGenerator` encapsulates many of the deatils of the explanation process. The class construct and explanation pipeline that is composed by the following steps:
- **Neighborhood generation**: the neighborhood of the instance to be explained is generated. The neighborhood is a synthetic dataset that is created by generating random instances around the instance to be explained. The neighborhood is used to understand the behavior of the blackbox model in the vicinity of the instance to be explained.
- ** Encoding**: the neighborhood dataset is encoded using one-hot encoding. The encoded neighborhood dataset is used to train the surrogate model.
- **Surrogate model**: a surrogate model is trained on the neighborhood dataset. The surrogate model is a simpler model that approximates the behavior of the blackbox model in the neighborhood of the instance to be explained. The surrogate model is used to extract classification rules that explain the behavior of the blackbox model.
- **Rule extraction**: the classification rules are extracted from the surrogate model. The classification rules are used to explain the classification of the instance to be explained. The classification rules provide insights into the features that are important for the classification of the instance.

Each of these step can be customized separately, in order to adapt the explanation process to the specific needs of the user. For example, let's construct an explanation process by managing manually the choice of each component of the explanation pipeline.

#### Encoding / Decoding

We use a `ColumnTransformerEnc` to exploit the `TransformerMixin` interface of the `sklearn` transformers. The `ColumnTransformerEnc` class is a wrapper around the `ColumnTransformer` class that provides the `encode` and `decode` methods to encode and decode the dataset. The `encode` method applies the transformations of the transformers to the dataset, while the `decode` method applies the inverse transformations to the dataset. The `ColumnTransformerEnc` class is used to encode and decode the dataset in the explanation process. 

In [8]:
from lore_sa.encoder_decoder import ColumnTransformerEnc

tabular_enc = ColumnTransformerEnc(dataset.descriptor)
ref_value = dataset.df.iloc[0].values[:-1]
encoded = tabular_enc.encode([ref_value])
decoded = tabular_enc.decode(encoded)

print(f"Original value: {ref_value}")
print(f"Encoded value: {encoded}")
print(f"Decoded value: {decoded}")

Original value: [39 'State-gov' 'Bachelors' 'Never-married' 'Adm-clerical' 'Not-in-family'
 'White' 'Male' 2174 0 40 'United-States']
Encoded value: [[39 2174 0 40 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0
  0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]]
Decoded value: [[39 'State-gov' 'Bachelors' 'Never-married' 'Adm-clerical'
  'Not-in-family' 'White' 'Male' 2174 0 40 'United-States']]


#### Neighborhood generation
Now that we are able to encode and decode the dataset, we can generate the neighborhood of the instance to be explained. To personalize the neighborhood generation process, we create now a Genetic Generator, using the class `GeneticGenerator`. The neighborhood is a synthetic dataset that is created by generating random instances around the instance to be explained, refined with a genetic algorithm to obtain a more dense and compact neighborhood.

In [9]:
from lore_sa.neighgen import RandomGenerator

num_row = 10
x = dataset.df.iloc[num_row][:-1]
z = tabular_enc.encode([x.values])[0] # remove the class feature from the input instance

gen = RandomGenerator(bbox=bbox, dataset=dataset, encoder=tabular_enc, ocr=0.1)
neighbour = gen.generate(z, 100, dataset.descriptor, tabular_enc)

print('Neighborhood', neighbour)

Neighborhood [[37 0 0 ... 1 0 0]
 [37 0 0 ... 1 0 0]
 [37 0 0 ... 1 0 0]
 ...
 [18.68399950796072 28428.28315499783 2751.6159728012435 ... 0 0 0]
 [18.68399950796072 28428.28315499783 2751.6159728012435 ... 0 0 0]
 [18.68399950796072 28428.28315499783 2751.6159728012435 ... 0 0 0]]


## surrogate model

The `surrogate` submodule creates a classifier and provides the methods to extract its corresponding classification rules. Once trained the surrogate, we extract a set of rules that explains why the surrogate model classifies the instance in a certain way. 

In the following example, the instance `x` has been classified as an adult with income less than 50k. The surrogate model has used the features `capital-gain`, `capital-loss`, `marital-status`, and `native-country`. 

In [10]:
from lore_sa.surrogate import DecisionTreeSurrogate
# decode the neighborhood to be labeled by the blackbox model
neighb_train_X = tabular_enc.decode(neighbour)
neighb_train_y = bbox.predict(neighb_train_X)
# encode the target class to the surrogate model
neighb_train_yz = tabular_enc.encode_target_class(neighb_train_y.reshape(-1, 1)).squeeze()

dt = DecisionTreeSurrogate()
x = dt.train(neighbour, neighb_train_yz)

In [11]:
num_row = 10
x = dataset.df.iloc[num_row][:-1] # remove the class feature from the input instance
z = tabular_enc.encode([x.values])[0]
rule = dt.get_rule(z, tabular_enc)
print('rule', rule)
crules, deltas = dt.get_counterfactual_rules(z, neighbour, neighb_train_yz, tabular_enc)
print('\n crules')
for c in crules:
    print(c)


rule premises:
capital-gain <= 33838.2412109375
occupation != Handlers-cleaners
capital-loss <= 3022.1815185546875
native-country != Columbia 
consequence: class = <=50K

 crules
premises:
capital-gain <= 33838.2412109375
occupation != Handlers-cleaners
capital-loss <= 3022.1815185546875
native-country = Columbia
sex != Male 
consequence: class = >50K
